## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt

# Import API key
from config1 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,67.86,94,0,2.30,clear sky
1,1,College,US,64.8569,-147.8028,60.78,60,75,4.61,smoke
2,2,Seoul,KR,37.5683,126.9778,76.39,94,100,14.97,heavy intensity rain
3,3,Kahului,US,20.8947,-156.4700,74.84,80,40,9.22,scattered clouds
4,4,Kruisfontein,ZA,-34.0033,24.7314,57.88,52,100,7.67,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Seoul,KR,37.5683,126.9778,76.39,94,100,14.97,heavy intensity rain
13,13,Neiafu,TO,-18.6500,-173.9833,76.32,67,99,15.93,overcast clouds
14,14,Samarai,PG,-10.6167,150.6667,79.75,82,100,16.06,overcast clouds
18,18,Saiki,JP,32.9500,131.9000,78.75,85,32,6.87,scattered clouds
22,22,Laguna,US,38.4210,-121.4238,79.02,43,0,9.22,clear sky
26,26,Bima,ID,-8.4667,118.7167,75.31,82,94,5.46,overcast clouds
27,27,Limbach-Oberfrohna,DE,50.8667,12.7500,85.44,36,6,7.56,clear sky
31,31,Georgetown,MY,5.4112,100.3354,84.88,77,20,5.75,moderate rain
33,33,Rikitea,PF,-23.1203,-134.9692,75.22,66,79,16.02,broken clouds
36,36,West Bay,KY,19.3667,-81.4167,79.00,84,16,1.99,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             243
City                   243
Country                242
Lat                    243
Lng                    243
Max Temp               243
Humidity               243
Cloudiness             243
Wind Speed             243
Current Description    243
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Seoul,KR,76.39,heavy intensity rain,37.5683,126.9778,
13,Neiafu,TO,76.32,overcast clouds,-18.6500,-173.9833,
14,Samarai,PG,79.75,overcast clouds,-10.6167,150.6667,
18,Saiki,JP,78.75,scattered clouds,32.9500,131.9000,
22,Laguna,US,79.02,clear sky,38.4210,-121.4238,
26,Bima,ID,75.31,overcast clouds,-8.4667,118.7167,
27,Limbach-Oberfrohna,DE,85.44,clear sky,50.8667,12.7500,
31,Georgetown,MY,84.88,moderate rain,5.4112,100.3354,
33,Rikitea,PF,75.22,broken clouds,-23.1203,-134.9692,
36,West Bay,KY,79.00,few clouds,19.3667,-81.4167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Seoul,KR,76.39,heavy intensity rain,37.5683,126.9778,롯데호텔 서울
13,Neiafu,TO,76.32,overcast clouds,-18.6500,-173.9833,Mystic Sands Beachfront Accommodation
14,Samarai,PG,79.75,overcast clouds,-10.6167,150.6667,Nuli Sapi
18,Saiki,JP,78.75,scattered clouds,32.9500,131.9000,（株）サンテツ
22,Laguna,US,79.02,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
26,Bima,ID,75.31,overcast clouds,-8.4667,118.7167,Lambitu Hotel
27,Limbach-Oberfrohna,DE,85.44,clear sky,50.8667,12.7500,"Landhotel ""Goldener Becher"" Limbach-oberfrohna"
31,Georgetown,MY,84.88,moderate rain,5.4112,100.3354,Cititel Penang
33,Rikitea,PF,75.22,broken clouds,-23.1203,-134.9692,People ThankYou
36,West Bay,KY,79.00,few clouds,19.3667,-81.4167,Calypso Cove


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "clean_hotel_df"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [17]:
clean_df.to_csv("WeatherPy_vacation.csv")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
!pip install gmaps
!jupyter nbextension enable --py gmaps
!jupyter nbextension enable --py widgetsnbextension


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))